# Проверка на простоту

*Число $m > 1$ является простым числом тогда и только тогда, когда
$$a^{m-1} \equiv 1 \mod m$$
для всех $a \not\equiv 0 \mod m$.*

Это следует с одной стороны из малой теоремы Ферма.
Если же $m$ составное, то есть $1 < a < m$, делящее $m$.
Предположим, что и в этом случае $a^{m-1} \equiv 1 \mod m$.
Тогда $a \mid m$ и $m \mid a^{m-1} - 1$.
Следовательно, $a \mid a^{m-1} - 1$, что значит $a k = a^{m-1} - 1$ для некоторого $k$ или $a^{m-1} - a k = 1$, т.е. $a \mid 1$.
Противоречие.

С помощью этого факта можно быстро доказать, что число составное.
Скажем, если $2^{m-1} \not\equiv 1 \mod m$, то число $m$ составное.

Но если для нескольких разных значений $a$, допустим, для всех $a$ взаимно простых с $m$, $\gcd(a,m)=1$, увидеть это не удалось, то возможно число $m$ простое?

Не обязательно (хотя и такой вероятностный тест применяют).
Существуют числа, которые проваливают такой "тест": например, $2^{1729-1} \equiv 1 \mod 1729$, но $1729 = 7 \cdot 13 \cdot 19$.

## Возведение в степень по модулю

Сначала научимся выполнять основную операцию для такой проверки.

**Задача.**
Напишите функцию `powmod(a,b,m)`, которая для целых $a,b$ и $m$ будет возвращать число $a^b \mod m$.

*Конечно, использовать встроенную функцию возведения в степень по модулю нельзя.*

Самый очевидный алгоритм, конечно,
```python
def powmod_easy(a, b, m):
    s = 1
    for i in xrange(b):
        s = (s * a) % m
    return s
```
Этот алгоритм выполняется за $O(b)$ операций умножения и взятия остатка.
Если $b$ велико, а на простоту проверяются обычно очень большие числа, то заниматься этим вычислением можно вечно.

Рассмотрим алгоритм сложность которого $O(\log b)$: понадобится всего $\log_2 b$ умножений по модулю.

Будем использовать двоичное представлени $b = (b_s b_{s-1} \ldots b_1 b_0)_2$ и искать 
$$(a^{2^0})^{b_0} \cdot (a^{2^1})^{b_1} \cdot (a^{2^2})^{b_2} \cdot \ldots \cdot (a^{2^s})^{b_s} \mod m = a^{b_0} \cdot (a^{2})^{b_1} \cdot ((a^{2})^2)^{b_2} \cdot \ldots \cdot ((a^{2^{s-1}})^{2})^{b_s} \mod m$$
в цикле.
На шаге $i = 0, 1, \ldots, s$ вычисляем $(a^{2^{i-1}})^2$ и просматриваем бит $b_i$.

**Задача.**
Проведите пару испытаний написанной функции `powmod`.
Cравните время выполнения с наивным примером `powermod_easy` (и при этом не повесьте компьютер), а также со встроенной `pow` или `power_mod`:
```python
timeit('powermod(2, 10^6 - 1, 10^6)')
```


## Числа Кармайкла

Числом Кармайкла (Carmichael) называется всякое составное число $m$, такое, что для всех $a$ взаимно простых с $m$ выполняется $a^{m-1} \equiv 1 \mod m$.

**Задача.**
Найдите самое маленькое число Кармайкла

а затем и

**Задача.**
Найдите все числа $m$ до некоторого $M$, такие, что для всех $a < m$ и $\gcd(a,m)=1$ выполнено $$a^{m-1} \equiv 1 \mod m$$